In [10]:
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz 
import os
import pickle
from matplotlib.font_manager import FontProperties
sns.set(font=['sans-serif'])
sns.set_style("whitegrid",{"font.sans-serif":['Microsoft JhengHei']})
# scikit-learn
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PowerTransformer, RobustScaler
from sklearn import datasets, linear_model, preprocessing, neighbors
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix,mean_absolute_error,mean_squared_error, r2_score
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,KFold,RandomizedSearchCV
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer
import xgboost as xgb
from sklearn.feature_selection import SequentialFeatureSelector

In [4]:
df_oh = pd.read_csv('../桃園市_predictset_onehot_test.csv', engine='python',index_col=[0])
df_oh

,交易年,總樓層數,主要建材,有無管理組織,車位總價元,房間數,建物移轉總面積坪,每坪價格,屋齡,車位,...,交易標的_房地(土地+建物),交易標的_房地(土地+建物)+車位,都市土地使用分區_住,都市土地使用分區_其他,都市土地使用分區_商,建物型態_住宅大樓(11層含以上有電梯),建物型態_公寓(5樓含以下無電梯),建物型態_套房(1房1廳1衛),建物型態_華廈(10層含以下有電梯),建物型態_透天厝
1,7,14,4,1,0.0,4,24.442000,19.024630,22,0,...,1,0,0,0,1,1,0,0,0,0
2,7,29,4,1,0.0,8,70.918100,29.329607,6,1,...,0,1,1,0,0,1,0,0,0,0
3,7,19,4,1,0.0,7,46.805825,17.519187,6,1,...,0,1,0,0,1,1,0,0,0,0
6,7,17,4,1,0.0,7,33.883025,15.494484,24,0,...,1,0,1,0,0,1,0,0,0,0
9,7,10,4,1,0.0,7,37.954675,24.502910,14,1,...,0,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511913,9,15,4,0,188.0,7,59.205300,33.375390,0,1,...,0,1,1,0,0,1,0,0,0,0
511914,9,15,4,0,198.0,7,59.235550,33.763509,0,1,...,0,1,1,0,0,1,0,0,0,0
511915,9,10,4,0,145.0,7,47.138575,20.959480,0,1,...,0,1,1,0,0,0,0,0,1,0
511916,9,10,4,0,140.0,5,35.419725,21.287574,0,1,...,0,1,1,0,0,0,0,0,1,0


## XGBRegressor

In [7]:
X=df_oh.drop(columns=['每坪價格','車位'])
y=df_oh['每坪價格']

In [8]:
# 調參前
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# 建立 XGBRegressor 模型
xgbr=xgb.XGBRegressor(tree_method="auto")
# xgbr=xgb.XGBClassifier(tree_method="auto",enable_categorical=True)
# 使用訓練資料訓練模型
xgbr.fit(X_train,y_train)

y_train_pred = xgbr.predict(X_train)
y_test_pred = xgbr.predict(X_test)

In [11]:
print(f'訓練集RMSE: {(mean_squared_error(y_train, y_train_pred,squared=False)):.3f},測試集: {(mean_squared_error(y_test, y_test_pred,squared=False)):.3f}')
print(f'訓練集MAE: {(mean_absolute_error(y_train, y_train_pred)):.3f},測試集: {(mean_absolute_error(y_test, y_test_pred)):.3f}')
print(f'訓練集R2: {(r2_score(y_train, y_train_pred)):.3f},測試集: {(r2_score(y_test, y_test_pred)):.3f}')

訓練集RMSE: 2.670,測試集: 2.853
訓練集MAE: 1.913,測試集: 2.022
訓練集R2: 0.795,測試集: 0.768


In [ ]:
importance=xgbr.feature_importances_
importance

In [ ]:
X_del=pd.DataFrame({'importance':xgbr.feature_importances_,'features':X.columns}).sort_values(by=['importance'],ascending=False)
mask_selected=X_del.iloc[0:20].features
X_selected=X_train[list(mask_selected)]
X_selected

In [ ]:
# grid 調參

params = {'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror']
              }

xgbclf = xgb.XGBRegressor(tree_method='gpu_hist',enable_categorical=True)
clf = RandomizedSearchCV(estimator=xgbclf,
                         param_distributions=params,
                         n_iter=10,
                         n_jobs=-1,
                         cv=3,
                         verbose=1)

clf.fit(X_train, y_train)

print(clf.best_params_,clf.best_score_)

In [12]:
# 調參後
# X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3)

# scaler = preprocessing.StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)

xgbr=xgb.XGBRegressor(tree_method="gpu_hist",enable_categorical=True,n_jobs=-1,
                      colsample_bytree=0.5,
                      min_child_weight=5,
                      subsample=0.7,
                      max_depth=11,
                      n_estimators=500,
                      learning_rate=0.03)
# 使用訓練資料訓練模型
xgbr.fit(X_train,y_train)

# X_test = scaler.transform(X_test)
y_train_pred = xgbr.predict(X_train)
y_test_pred = xgbr.predict(X_test)

In [13]:
print(f'訓練集RMSE: {(mean_squared_error(y_train, y_train_pred,squared=False)):.3f},測試集: {(mean_squared_error(y_test, y_test_pred,squared=False)):.3f}')
print(f'訓練集MAE: {(mean_absolute_error(y_train, y_train_pred)):.3f},測試集: {(mean_absolute_error(y_test, y_test_pred)):.3f}')
print(f'訓練集R2: {(r2_score(y_train, y_train_pred)):.3f},測試集: {(r2_score(y_test, y_test_pred)):.3f}')

訓練集RMSE: 2.102,測試集: 2.534
訓練集MAE: 1.456,測試集: 1.702
訓練集R2: 0.873,測試集: 0.817


In [ ]:
kf =KFold(n_splits=5, shuffle=True, random_state=42)
sc=cross_val_score(model,X, y, cv= kf)

In [ ]:
sc.mean()

In [14]:
# Saving model
with open("model_XGBR_taoyuan.pickle","wb") as f:
    pickle.dump(xgbr, f, protocol=pickle.HIGHEST_PROTOCOL)